In [125]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l


In [126]:
# 构造一个数据集 ,好处是 知道真实的w和b
# w = [2,5.6] 、 b = 1.8 和噪声项c生成数据集和标签
# y = Xw + b + c

def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

def line(x,w,b):
    return torch.matmul(x, w) + b

def sqare_loss(y_hat,y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2
def load_data( features, labels,batch_size):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


In [127]:
def train_data():
    # W = torch.tensor([2,5.6])
    # b = 1.8
    # features,labels = synthetic_data(W,b,1000)
    # return features,labels
    true_w = torch.tensor([2, -3.4])
    true_b = 4.2
    features, labels = synthetic_data(true_w, true_b, 1000)
    return features,labels


In [141]:
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [144]:
def train():
    # 定义一些超参数
    batch_size = 10
    lr  = 0.02
    net = line
    loss = sqare_loss
    epochs = 5
    # 获取之前创建的数据
    features,labels = train_data()
    data_loader = load_data(features,labels,batch_size)
    # 创建线形层的参数
    w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)

    #开始每一层的训练
    for epoch in range(epochs):
        # 这里一定要使用函数load_data，因为要保证每次加载的数据不一样，如果一样了，那么梯度没变化
        for X,y in load_data(features,labels,batch_size):
            l = loss(net(X,w,b),y)
            l.sum().backward()
            sgd([w,b],lr,batch_size)
            
        with torch.no_grad():
            test_loss = loss(net(features,w,b),labels)
            print(f"第 {epoch+1}次 进行训练，损失值是： {test_loss.mean()}")
            


In [145]:
train()

第 1次 进行训练，损失值是： 1.6128489971160889
第 2次 进行训练，损失值是： 6.294341564178467
第 3次 进行训练，损失值是： 29.576749801635742
第 4次 进行训练，损失值是： 5.830108165740967
第 5次 进行训练，损失值是： 4.598476886749268
